In [1]:
import torch
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline

matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# Classes create for preprocessing and models
from preprocess.preprocess import MovieLens
from models.matrix_factorization import MF
from rec_sys.recsys import RecSys

In [3]:
ml = MovieLens()

In [4]:
n_users, n_items = ml.num_user_item()
model = MF(n_users, n_items, emb_dim=128, init=True)
model.to(device)
print(model)

In [5]:
train_loader, test_loader = ml.dataset(batch_size=32)

In [13]:
train_losses, val_losses = model.fit(num_epochs=20, train_loader=train_loader, test_loader=test_loader)

In [14]:
plt.figure(figsize=(8, 4))
plt.plot(train_losses, label='Train')
plt.plot(val_losses, label='Val')
plt.title('Loss vs Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid()
plt.legend();

In [15]:
user_emb_min_w = model.user_emb.weight.min().item()
user_emb_max_w = model.user_emb.weight.max().item()
item_emb_min_w = model.item_emb.weight.min().item()
item_emb_max_w = model.item_emb.weight.max().item()

print(f'Emb user min/max w: {user_emb_min_w:0.3f} / {user_emb_max_w:0.3f}')
print(f'Emb item min/max w: {item_emb_min_w:0.3f} / {item_emb_max_w:0.3f}')
# print(f'Preds min/max: {y_ratings.min().item():0.2f} / {y_hat.max().item():0.2f}')
# print(f'Rating min/max: {yRatings.min().item():0.2f} / {yRatings.max().item():0.2f}')
# print(preds.detach().cpu().numpy()[:6])
# print(y_ratings.detach().cpu().numpy()[:6])

In [20]:
matrix = ml.get_matrix()
movies = ml.get_movies()
matrix.shape

In [21]:
user_ids, item_ids = ml.get_ids()

user_id = 5
rec_sys = RecSys(model, matrix, user_ids, item_ids, movies)
list_movies = rec_sys.make_recommendations(user_id)


In [22]:
print("Rank of recommendations for user: ", user_id)
for movie in list_movies:    
    print("Title: " + movie['movie_name'] + " Rating: " + str(movie['predicted_rating'].detach().cpu().numpy()))

In [23]:
rmse = model.calculate_rmse(test_loader)
print(f"RMSE: {rmse}")

In [ ]:
print('test')